In [3]:
# %autoreload 1 only needs to be run once after each kernel start
%load_ext autoreload
%autoreload 2
# %aimport MelSpectrogramConfig, AudioSample   


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%reset -f

In [4]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchaudio
from torchaudio.transforms import AmplitudeToDB, GriffinLim
import torchaudio.transforms as T
from IPython.display import Audio, display
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from AudioSample import AudioSample
from MelSpectrogramConfig import MelSpectrogramConfig
import config as C
from config import BASE_PATH, DEFAULT_DIRECTORY, DEFAULT_FILE_INDEX, DEFAULT_THRESHOLD
from widgets_setup import create_widgets
from display_functions import display_audio_in_widget, plot_audio_waveform_in_widget, plot_mel_spectrogram_in_widget, plot_spikes_in_widget
from widget_sync_utils import set_audio_sample_from_widget_values, set_mel_config_from_widget_values
from Spikes import Spikes

# Initialize AudioSample and MelSpectrogramConfig
audio_sample = AudioSample(BASE_PATH, DEFAULT_DIRECTORY, DEFAULT_FILE_INDEX)
mel_config = MelSpectrogramConfig(audio_sample, n_fft=C.DEFAULT_N_FFT, hop_length=C.DEFAULT_HOP_LENGTH, n_mels=C.DEFAULT_N_MELS, f_min=C.DEFAULT_F_MIN, f_max=C.DEFAULT_F_MAX, power=C.DEFAULT_POWER, filter_type="custom", toggle_mel_filter="spktgrm")
spikes_data = Spikes(threshold=C.DEFAULT_THRESHOLD)

# Initialize widgets
(
    directory_dropdown, 
    file_slider, 
    n_fft_input, 
    hop_length_slider, 
    n_mels_slider, 
    f_min_slider, 
    f_max_slider, 
    power_toggle, 
    threshold_slider, 
    spike_plot_radio, 
    spike_periode_slider,
    spike_period_slider_combo,
    spk_freq_label,
    channel_slider,
    filter_choice_radio,
    mel_plot_radio
) = create_widgets(audio_sample, mel_config, spikes_data)

# Output widgets
output_play = widgets.Output(layout=widgets.Layout())
output_audio_signal = widgets.Output(layout=widgets.Layout())
output_melspecrogram = widgets.Output(layout=widgets.Layout())
output_spikes = widgets.Output(layout=widgets.Layout())

# this func. used for test purpose:
def beep_sound(message):
    print(f'Beep function called {message}')  # Confirm it's called
    frequency = 440  # Hz
    duration = 0.2   # seconds
    sample_rate = 44100
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    wave = 0.5 * np.sin(2 * np.pi * frequency * t)
    display(Audio(wave, rate=sample_rate, autoplay=True))

# Observers for widget changes
def update_plot(change):
    set_audio_sample_from_widget_values(audio_sample, directory_dropdown, file_slider)
    display_audio_in_widget(audio_sample, output_play)
    plot_audio_waveform_in_widget(audio_sample, output_audio_signal)
    update_plot_mel(None)

def update_plot_mel(change):
    set_mel_config_from_widget_values(mel_config, n_fft_input, hop_length_slider, n_mels_slider, f_min_slider, f_max_slider, power_toggle, filter_choice_radio, mel_plot_radio)
    plot_mel_spectrogram_in_widget(audio_sample, mel_config, output_melspecrogram)
    update_plot_spike(None)
    channel_slider.max=mel_config.n_mels
    
def update_plot_spike(change):
    spikes_data.threshold = threshold_slider.value
    plot_radio = spike_plot_radio.value
    plot_spikes_in_widget(audio_sample, mel_config, spikes_data, plot_radio, output_spikes)


def check_filter_type(change):
    """Set widget values and disable state based on filter choice."""
    if filter_choice_radio.value == "custom":
        # Set custom filter parameters
        f_min_slider.value = 275
        f_max_slider.value = 7625
        n_mels_slider.value = 15
        # Disable sliders for custom filter
        f_min_slider.disabled = True
        f_max_slider.disabled = True
        n_mels_slider.disabled = True
    else:
        # Enable sliders for standard filter
        f_min_slider.disabled = False
        f_max_slider.disabled = False
        n_mels_slider.disabled = False

    # Trigger the update for the spectrogram plot
    update_plot_mel(change)
    

# Attach observers
directory_dropdown.observe(update_plot, names='value')
file_slider.observe(update_plot, names='value')
n_fft_input.observe(update_plot_mel, names='value')
hop_length_slider.observe(update_plot_mel, names='value')
n_mels_slider.observe(update_plot_mel, names='value')
f_min_slider.observe(update_plot_mel, names='value')
f_max_slider.observe(update_plot_mel, names='value')
power_toggle.observe(update_plot_mel, names='value')
threshold_slider.observe(update_plot_spike, names='value')
spike_plot_radio.observe(update_plot_spike, names='value')
filter_choice_radio.observe(check_filter_type, names='value')
mel_plot_radio.observe(update_plot_mel, names='value')

# Initial display setup
update_plot(None)  # Trigger initial display updates
update_plot_mel(None)
update_plot_spike(None)
# update_spike_info(None)

# # Layout and display
# Layout definiálása a középre igazításhoz
centered_layout = widgets.Layout(align_items='center', justify_content='center')
align_top_layout = widgets.Layout(align_items='flex-start', justify_content='flex-start')

# widget groups
file_controls = widgets.VBox([directory_dropdown, file_slider, output_play], layout=centered_layout)
mel_spectrogram_controls = widgets.VBox([hop_length_slider, n_mels_slider, f_min_slider, f_max_slider, n_fft_input, power_toggle, filter_choice_radio, mel_plot_radio], layout=centered_layout)
spike_control = widgets.VBox([threshold_slider, spike_plot_radio, spike_period_slider_combo], layout=centered_layout)
dummy_widget = widgets.Box(layout=widgets.Layout(width=file_controls.layout.width, height=file_controls.layout.height))

# main row arrangment with HBox creating horizonatl groups
control_layout = widgets.HBox([file_controls, mel_spectrogram_controls, spike_control], layout=widgets.Layout(justify_content='space-around'))

# widget groups
audio_output = widgets.VBox([output_audio_signal], layout=align_top_layout)
mel_spectrogram_output = widgets.VBox([output_melspecrogram], layout=align_top_layout)
spike_output = widgets.VBox([output_spikes], layout=align_top_layout)

# main row arrangment with HBox creating horizonatl groups
figure_layout = widgets.HBox([audio_output, mel_spectrogram_output,spike_output], layout=widgets.Layout(justify_content='space-around'))

# Megjelenítés
display(control_layout)
display(figure_layout)

In [6]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])